In [1]:
import pandas as pd
from os import walk, makedirs
from os.path import join, exists
import openpyxl as XL
from tqdm import tqdm_notebook
import xml.etree.ElementTree as elemTree
import os, operator
from os.path import getsize
from tqdm.notebook import tqdm as tn

In [5]:

def get_data(filename):
    tree = elemTree.parse(filename)
    root = tree.getroot()
    basic_data = root.find('BestExRptgRTS27DataRpt').find('Header')

    VnNm = basic_data.find('VnNm').text
    VnCd = basic_data.find('VnCd').text
    CtryCompAuth = basic_data.find('CtryCompAuth').text
    MktSgmt = basic_data.find('MktSgmt').text
    MktSgmtNm = basic_data.find('MktSgmtNm').text
    DtTrdDay = basic_data.find('DtTrdDay').text

    basic_data = {
        'VnNm': [VnNm],
        'VnCd': [VnCd],
        "CtryCompAuth": [CtryCompAuth],
        'MktSgmt': [MktSgmt],
        'MktSgmtNm': [MktSgmtNm],
        'DtTrdDay': [DtTrdDay]
    }
    columns = "VnNm VnCd CtryCompAuth MktSgmt MktSgmtNm DtTrdDay " \
              "FinInstr FinInstrNm CFICd Ccy " \
              "Data SzRn Pric ExecTime TxSize TrdSystm TrdMode TrdPlt " \
              "SmplAvgExecPric TtlValExec " \
              "SmplAvgTxPric VolWghTxPric HgstTxPric LwstTxPric " \
              "NumRcvdOrQt NumExecTx TtlValExecTx NmOfCxlQrQt " \
              "NmOfAmndQrQt MdnTxSize MdnOrQtSize " \
              "Data2 BstBdPric BstOfrPric BbSz OfrSz".split(" ")

    all_data = pd.DataFrame(columns=columns)

    for fininst in root.find('BestExRptgRTS27DataRpt').find('Body').iter('FinInstrument'):
        fin_basic = {
            "FinInstr": [fininst.find('FinInstr').text],
            "FinInstrNm": [fininst.find('FinInstrNm').text],
            "CFICd": [fininst.find('CFICd').text],
            "Ccy": [fininst.find('Ccy').text]
        }
        DaylyPric = fininst.find('DaylyPric')
        if DaylyPric is not None:
            for data in DaylyPric.iter():
                if data.tag != 'DaylyPric':
                    fin_basic[data.tag] = data.text

        LkhdExec = fininst.find('LkhdExec') # NumRcvdOrQt etc...
        if LkhdExec is not None:
            for data in LkhdExec.iter():
                if data.tag != "LkhdExec":
                    fin_basic[data.tag] = data.text

        IntrDayPric = fininst.find('IntrDayPric') # SzRn etc...
        if IntrDayPric is not None:
            for Time in IntrDayPric.iter('Time'):
                time_data = {}
                time_data['SzRn'] = Time.find('Data').attrib['SzRn']
                time_data['Data'] = Time.find('Data').text
                FrstTrd = Time.find('FrstTrd')

                if FrstTrd is not None:
                    frsttrd_data = {}
                    for data in FrstTrd.iter():
                        if data.tag != "FrstTrd":
                            frsttrd_data[data.tag] = data.text

                    all_data = all_data.append(pd.DataFrame.from_dict(
                        {**frsttrd_data, **time_data, **fin_basic, **basic_data}),
                        ignore_index=True)

                AllTrds = Time.find('AllTrds')

                if AllTrds is not None:
                    alltrds_data = {
                        'SmplAvgExecPric': AllTrds.find('SmplAvgExecPric').text,
                        'TtlValExec': AllTrds.find('TtlValExec').text
                    }
                    all_data = all_data.append(pd.DataFrame.from_dict(
                        {**alltrds_data, **time_data, **fin_basic, **basic_data}
                    ), ignore_index=True)

        LkhdExecInfoTm = fininst.find('LkhdExecInfoTm')
        if LkhdExecInfoTm is not None:
            for Time in LkhdExecInfoTm.iter('Time'):
                time_data = {}
                for data in Time.iter():
                    if data.tag == 'Data':
                        time_data['Data2'] = data.text
                    elif data.tag != 'Time':
                        time_data[data.tag] = data.text

                all_data = all_data.append(pd.DataFrame.from_dict(
                        {**time_data, **fin_basic, **basic_data}),
                        ignore_index=True)

        # if LkhdExecInfoTm.find('Time') is None and IntrDayPric is None:
        if LkhdExecInfoTm.find('Time') is None:

            all_data = all_data.append(pd.DataFrame.from_dict(
                {**fin_basic, **basic_data}), ignore_index=True)

    return all_data

def write_data(filepath, filename, all_data, header=False, mode='a'):
    full_filename = join(filepath, filename)
    all_data.to_csv(
        full_filename,
        index=False,
        sep=",",
        mode=mode,
        header=header
    )

def make_new(save_num):
    SAVE_PATH = "/home/genne/Downloads/save/new"
    save_name = f"bond_transactions_{save_num}.csv"
    full_filename = join(SAVE_PATH, save_name)
    columns = "VnNm VnCd CtryCompAuth MktSgmt MktSgmtNm DtTrdDay " \
          "FinInstr FinInstrNm CFICd Ccy " \
          "Data SzRn Pric ExecTime TxSize TrdSystm TrdMode TrdPlt " \
          "SmplAvgExecPric TtlValExec " \
          "SmplAvgTxPric VolWghTxPric HgstTxPric LwstTxPric " \
          "NumRcvdOrQt NumExecTx TtlValExecTx NmOfCxlQrQt " \
          "NmOfAmndQrQt MdnTxSize MdnOrQtSize " \
          "Data2 BstBdPric BstOfrPric BbSz OfrSz".split(" ")
    all_data = pd.DataFrame(columns=columns)

    all_data.to_csv(full_filename, index=False)

    return save_name


In [6]:
PATH = "/home/genne/Downloads/2020/files" # need to modify this
files_list = []
for path, dirs, files in os.walk(PATH):
    files_list.extend([(os.path.join(path, file),
                        getsize(os.path.join(path, file))) for file in files])
files_list = sorted(files_list, key=operator.itemgetter(1), reverse=False)
print(len(files_list))

366


In [7]:
SAVE_PATH = "/home/genne/Downloads/save/new"
SAVE_PATH_SEP = "/home/genne/Downloads/2020/new"
save_name = 'bond_transactions_1.csv'
for fname, fsize in tqdm_notebook(files_list):
    all_data = get_data(fname)
    write_data(SAVE_PATH, save_name, all_data)
    write_data(SAVE_PATH_SEP, os.path.basename(fname).split(".")[0]+".csv", all_data, mode='w', header=True)

<ipython-input-7-7337b205e7fe>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fname, fsize in tqdm_notebook(files_list):


  0%|          | 0/366 [00:00<?, ?it/s]

In [39]:
import os, operator
from os.path import getsize
files_list = []
for path, dirs, files in os.walk(PATH):
    files_list.extend([(os.path.join(path, file),
                        getsize(os.path.join(path, file))) for file in files])
files_list = sorted(files_list, key=operator.itemgetter(1), reverse=True)
print(files_list)

[('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180612_DBES.xml', 20702926), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180206_DBES.xml', 19134167), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180611_DBES.xml', 18575048), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180209_DBES.xml', 18455259), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180613_DBES.xml', 18453728), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180207_DBES.xml', 18147585), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180214_DBES.xml', 18002793), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180205_DBES.xml', 17908132), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180208_DBES.xml', 17882973), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180202_DBES.xml', 17761478), ('/home/genne/Downloads/output/RTS27_7LTWFZYICNSX8D621K86_20180302_DBES.xml', 17711198), ('/home/genne/Downlo

In [40]:
SAVE_PATH = "/home/genne/Downloads/transactions_new"
save_name = make_new(0)
idx = 0
notebook_idx = 1
fsize_sum = 0
for fname, fsize in tqdm_notebook(files_list):
    if fsize_sum > 50000000:
        save_name = make_new(notebook_idx) #file's too big -> rows overflow after 1000000
        notebook_idx += 1
        fsize_sum = 0
    fsize_sum += fsize
    all_data = get_data(fname)
    write_data(SAVE_PATH, save_name, all_data)
    idx += 1

<ipython-input-40-58f2c6218b0d>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fname, fsize in tqdm_notebook(files_list):


  0%|          | 0/1023 [00:00<?, ?it/s]

In [17]:
SAVE_TWO = "/home/genne/Downloads/excel"
dirpath, dirnames, filenames = next(walk(SAVE_PATH))
print(filenames)
for fname in tqdm_notebook(filenames):
    full_path = join(dirpath, fname)
    csv_data = pd.read_csv(full_path)
    excel_path = join(SAVE_TWO, fname.split(".")[0]+".xlsx")
    with pd.ExcelWriter(excel_path, mode='w') as writer:
        csv_data.to_excel(writer, index=False)

['bond_transactions_0.csv', 'bond_transactions_7.csv', 'bond_transactions_6.csv', 'bond_transactions_2.csv', 'bond_transactions_4.csv', 'bond_transactions_5.csv', 'bond_transactions_9.csv', 'bond_transactions_1.csv', 'bond_transactions_10.csv', 'bond_transactions_8.csv', 'bond_transactions_3.csv']


<ipython-input-17-69fe4a723b28>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fname in tqdm_notebook(filenames):


  0%|          | 0/11 [00:00<?, ?it/s]

In [23]:
import os, operator
from os.path import getsize
from tqdm.notebook import tqdm as tn

TOTAL_PATH = "/home/genne/Downloads"
save_name = "bond_transactions.csv"
CSV_PATH = "/home/genne/Downloads/transactions_new"

files_list = []
for path, dirs, files in os.walk(CSV_PATH):
    files_list.extend([(os.path.join(path, file),
                        getsize(os.path.join(path, file))) for file in files
                       if file.endswith(".csv")])
files_list = sorted(files_list, key=operator.itemgetter(1), reverse=True)
print(files_list[:2])

[('/home/genne/Downloads/transactions_new/bond_transactions_100.csv', 41213214), ('/home/genne/Downloads/transactions_new/bond_transactions_103.csv', 33419857)]


In [24]:
for csv_file, filesize in tn(files_list):
    csv_data = pd.read_csv(csv_file)
    columns = "VnNm VnCd CtryCompAuth MktSgmt MktSgmtNm DtTrdDay " \
          "FinInstr FinInstrNm CFICd Ccy " \
          "Data SzRn Pric ExecTime TxSize TrdSystm TrdMode TrdPlt " \
          "SmplAvgExecPric TtlValExec " \
          "SmplAvgTxPric VolWghTxPric HgstTxPric LwstTxPric " \
          "NumRcvdOrQt NumExecTx TtlValExecTx NmOfCxlQrQt " \
          "NmOfAmndQrQt MdnTxSize MdnOrQtSize " \
          "Data2 BstBdPric BstOfrPric BbSz OfrSz".split(" ")
    csv_data = csv_data[2:]
    csv_data.columns = columns
    csv_data.to_csv(
        join(TOTAL_PATH, save_name),
        index=False,
        sep=",",
        mode='a',
        header="False"
    )

  0%|          | 0/102 [00:00<?, ?it/s]

/home/genne/Downloads/transactions_new/bond_transactions_100.csv


ParserError: Error tokenizing data. C error: Expected 36 fields in line 2, saw 40


In [18]:
print(files_list[-1])
csv_data = pd.read_csv(files_list[-1][0])
columns = "VnNm VnCd CtryCompAuth MktSgmt MktSgmtNm DtTrdDay " \
      "FinInstr FinInstrNm CFICd Ccy " \
      "Data SzRn Pric ExecTime TxSize TrdSystm TrdMode TrdPlt " \
      "SmplAvgExecPric TtlValExec " \
      "SmplAvgTxPric VolWghTxPric HgstTxPric LwstTxPric " \
      "NumRcvdOrQt NumExecTx TtlValExecTx NmOfCxlQrQt " \
      "NmOfAmndQrQt MdnTxSize MdnOrQtSize " \
      "Data2 BstBdPric BstOfrPric BbSz OfrSz".split(" ")
csv_data = csv_data[1:5]
csv_data.columns = columns
print(csv_data)

('/home/genne/Downloads/transactions_new/bond_transactions_93.csv', 6085082)
                                                                                                            VnNm  \
DEUTSCHE BANK AKTIENGESELLSCHAFT 7LTWFZYICNSX8D621K86 DE DBES  DEUTSCHE BANK AG OPERATING MARKET CODE FOR EU ...   
                                                         DBES  DEUTSCHE BANK AG OPERATING MARKET CODE FOR EU ...   
                                                         DBES  DEUTSCHE BANK AG OPERATING MARKET CODE FOR EU ...   
                                                         DBES  DEUTSCHE BANK AG OPERATING MARKET CODE FOR EU ...   

                                                                     VnCd  \
DEUTSCHE BANK AKTIENGESELLSCHAFT 7LTWFZYICNSX8D621K86 DE DBES  2019-08-21   
                                                         DBES  2019-08-21   
                                                         DBES  2019-08-21   
                                  

In [30]:
csv_data = get_data("/home/genne/Downloads/sample.xml")
csv_data.to_csv(
    "/home/genne/Downloads/test2.csv",
    index=False,
    sep=","
)